# Building a Comedy Sketch Creation System with Moya

This notebook demonstrates how to create a multi-agent comedy sketch creation system using Moya for the multi-agent framework with OpenAI agents.

## Installation

First, let's install the required packages.

In [3]:
import os
os.environ["CHROMADB_CLIENT"] = "true"

## Import Required Libraries

In [4]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

In [5]:
from moya.agents.openai_agent import OpenAIAgent, OpenAIAgentConfig
from moya.agents.remote_agent import RemoteAgent, RemoteAgentConfig
from moya.classifiers.llm_classifier import LLMClassifier
from moya.orchestrators.multi_agent_orchestrator import MultiAgentOrchestrator
from moya.registry.agent_registry import AgentRegistry
from moya.tools.ephemeral_memory import EphemeralMemory
from moya.memory.in_memory_repository import InMemoryRepository
from moya.tools.tool_registry import ToolRegistry

In [6]:
# Load environment variables from .env file
load_dotenv()

# Get Azure OpenAI credentials
azure_api_key = os.environ.get("AZURE_azure_api_key")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

# Define Azure deployment name - you'll need to replace this with your actual deployment name
deployment_name = "gpt-4o"  # Replace with your Azure deployment name

# Create an Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=azure_api_key,
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)

# Verify the environment variables are loaded
print("Azure OpenAI API configuration loaded from .env file")

Azure OpenAI API configuration loaded from .env file


## Create Agent Registry and Tool Registry

Setting up the foundation for our multi-agent system.

In [7]:
# Create a tool registry to manage tools that agents can use
tool_registry = ToolRegistry()

# Add EphemeralMemory tool to allow agents to remember information
EphemeralMemory.configure_memory_tools(tool_registry)

# Create an agent registry to manage our agents
agent_registry = AgentRegistry()

## Define OpenAI Agents

We'll create five comedy sketch creation agents with different roles.

In [8]:
# Set up API key - replace with your actual OpenAI API key
azure_api_key = os.getenv("azure_api_key", "your-api-key-here")

# Create a Premise & Concept Generator agent
concept_generator_config = OpenAIAgentConfig(
    agent_name="concept_generator",
    agent_type="openai",
    description="Proposes unique comedic premises, absurd scenarios, and humorous character ideas based on themes or genres.",
    system_prompt="""You are an expert comedy concept generator. Your specialty is creating fresh, 
    engaging comedy sketch premises that align with user preferences. You excel at developing 
    absurd scenarios, unique character concepts, and innovative comedic situations. Focus on 
    originality while ensuring the concepts have strong comedic potential.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
concept_generator = OpenAIAgent(concept_generator_config)
concept_generator.client = azure_client

# Create a Dialogue & Punchline Refinement agent
dialogue_refiner_config = OpenAIAgentConfig(
    agent_name="dialogue_refiner",
    agent_type="openai",
    description="Suggests witty dialogue, one-liners, and comedic callbacks while ensuring proper setup and delivery.",
    system_prompt="""You are a skilled comedy dialogue writer who excels at crafting witty dialogue, 
    memorable one-liners, and effective comedic callbacks. Your expertise is in enhancing timing, 
    phrasing, and impact to make sketches entertaining and memorable. Ensure proper setup and delivery 
    for maximum comedic effect.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
dialogue_refiner = OpenAIAgent(dialogue_refiner_config)
dialogue_refiner.client = azure_client

# Create a Sketch Structure & Timing agent
structure_timing_config = OpenAIAgentConfig(
    agent_name="structure_timing",
    agent_type="openai",
    description="Organizes comedic escalation, beats, and timing cues to create well-paced sketches.",
    system_prompt="""You are an expert in comedy sketch structure and timing. You excel at organizing 
    comedic escalation, beats, and timing cues to create well-paced sketches. Your role is to ensure 
    that sketches flow naturally and land effectively, avoiding pacing issues and maximizing comedic impact. 
    Provide clear guidance on sketch structure, including intros, escalation points, and satisfying endings.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
structure_timing = OpenAIAgent(structure_timing_config)
structure_timing.client = azure_client

# Create an Audience Adaptation & Cultural Context agent
audience_adaptation_config = OpenAIAgentConfig(
    agent_name="audience_adaptation",
    agent_type="openai",
    description="Adjusts tone, references, and comedic elements to suit different audiences, platforms, or performance styles.",
    system_prompt="""You are a specialist in tailoring comedy for specific audiences. Your expertise is 
    in adjusting tone, references, and comedic elements to suit different audiences, platforms, or 
    performance styles. You make material relatable and impactful for specific demographics or regions 
    while maintaining the core comedic elements of the sketch.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
audience_adaptation = OpenAIAgent(audience_adaptation_config)
audience_adaptation.client = azure_client

# Create a Collaborative Brainstorming & Variation agent
variation_agent_config = OpenAIAgentConfig(
    agent_name="variation_agent",
    agent_type="openai",
    description="Provides alternate takes, subversions of tropes, and unexpected comedic twists to avoid predictable humor.",
    system_prompt="""You are a creative comedy brainstormer who specializes in providing alternate takes, 
    subversions of tropes, and unexpected comedic twists. Your goal is to help avoid predictable humor 
    by encouraging creativity, spontaneity, and uniqueness in sketch writing. Offer multiple variations 
    and surprising directions to take comedy sketches.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
variation_agent = OpenAIAgent(variation_agent_config)
variation_agent.client = azure_client

# Register all agents with the agent registry
agent_registry.register_agent(concept_generator)
agent_registry.register_agent(dialogue_refiner)
agent_registry.register_agent(structure_timing)
agent_registry.register_agent(audience_adaptation)
agent_registry.register_agent(variation_agent)

## Create a Classifier

We need a classifier to route messages to the appropriate agent.

In [9]:
# Reference:
# class LLMClassifier(BaseClassifier):
#     """LLM-based classifier for agent selection."""

#     def __init__(self, llm_agent: Agent, default_agent: str):
#         """
#         Initialize with an LLM agent for classification.
        
#         :param llm_agent: An agent that will be used for classification
#         :param default_agent: The default agent to use if no specialized match is found
#         """
#         self.llm_agent = llm_agent
#         self.default_agent = default_agent





# Create a classifier agent for routing messages
classifier_agent_config = OpenAIAgentConfig(
    agent_name="classifier",
    agent_type="openai",
    description="Routes messages to the appropriate specialized agent",
    system_prompt="You are a classifier that determines which specialized agent should handle a given request.",
    model_name="gpt-4o",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
classifier_agent = OpenAIAgent(classifier_agent_config)
classifier_agent.client = azure_client

# Create an LLM-based classifier to route messages to appropriate agents
classifier = LLMClassifier(
    llm_agent=classifier_agent,
    default_agent="concept_generator"  # Using concept_generator as default agent
)

## Set up Multi-Agent Orchestrator

Now we'll create a multi-agent orchestrator to manage the interactions between agents.

In [10]:
# Create a multi-agent orchestrator to manage interactions
orchestrator = MultiAgentOrchestrator(
    agent_registry=agent_registry,
    classifier=classifier
)

## Define Multi-Agent Workflow

Let's create a workflow for our comedy sketch creation project.

In [11]:
# Create the initial user request
user_request = "I need a comedy sketch about technology addiction in modern society, suitable for a YouTube audience."

# Create a thread ID for this conversation
thread_id = "comedy_sketch_001"

# Store user request in memory properly using EphemeralMemory directly
EphemeralMemory.store_message(thread_id=thread_id, sender="user", content=user_request)
print(f"Stored initial user request in memory")

# Define the concept generation task
concept_task = """Generate 3 unique comedy sketch concepts about technology addiction in modern society. 
Each concept should include a brief premise, key characters, and the central comedic tension. 
Make the concepts fresh, engaging, and suitable for a YouTube audience."""

Stored initial user request in memory


In [12]:
# Get conversation summary from memory
summary_tool = tool_registry.get_tool("get_summary")
if summary_tool:
    summary = summary_tool.function(thread_id=thread_id)
    print(f"Conversation summary:\n{summary}\n")


Conversation summary:
Summary of thread comedy_sketch_001:
user said: I need a comedy sketch about technology addiction in modern society, suitable for a YouTube audience.



## Collaborative Problem Solving

Let's also demonstrate how the agents can work together to solve a comedy challenge interactively.

In [11]:
# Create a new thread for this collaborative session
collab_thread_id = "comedy_collab_001"

# Start with a comedy challenge that requires collaborative effort
initial_challenge = """Create a comedy sketch that combines physical comedy with social commentary 
about social media influence. It should be appropriate for a live theater audience."""

# Store the challenge in memory
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="user", content=initial_challenge)
print("Stored initial challenge in memory\n")

# First, let the concept generator propose ideas
print("Collaborative Session - Concept Generator's Turn\n")
concept_thoughts = orchestrator.orchestrate(
    thread_id=collab_thread_id,
    user_message=initial_challenge,
    agent_name="concept_generator"
)
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="concept_generator", content=concept_thoughts)
print(f"Concept Generator: {concept_thoughts}\n")

# Dialogue refiner builds on concept generator's input
dialogue_prompt = "Based on the concept generator's ideas, create compelling dialogue for this sketch about social media influence."
print("Collaborative Session - Dialogue Refiner's Turn\n")

# Get conversation summary from memory
summary_tool = tool_registry.get_tool("get_summary")
if summary_tool:
    summary = summary_tool.function(thread_id=collab_thread_id)

dialogue_contribution = orchestrator.orchestrate(
    thread_id=collab_thread_id,
    user_message=dialogue_prompt + "\n\nPrevious contributions: " + summary,
    agent_name="dialogue_refiner"
)
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="dialogue_refiner", content=dialogue_contribution)
print(f"Dialogue Refiner: {dialogue_contribution}\n")

# Structure agent organizes everything
structure_prompt = "Based on the concept and dialogue provided, create a structured outline for this social media sketch."

print("Collaborative Session - Structure & Timing Agent's Turn\n")

# Get updated conversation summary
if summary_tool:
    summary = summary_tool.function(thread_id=collab_thread_id)

structure_contribution = orchestrator.orchestrate(
    thread_id=collab_thread_id,
    user_message=structure_prompt + "\n\nPrevious contributions: " + summary,
    agent_name="structure_timing"
)
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="structure_timing", content=structure_contribution)
print(f"Structure & Timing Agent: {structure_contribution}\n")

# Audience adaptation agent tailors it for live theater
adaptation_prompt = "Tailor this sketch specifically for a live theater audience, highlighting elements that would work well in that medium."

print("Collaborative Session - Audience Adaptation Agent's Turn\n")

# Get updated conversation summary
if summary_tool:
    summary = summary_tool.function(thread_id=collab_thread_id)

adaptation_contribution = orchestrator.orchestrate(
    thread_id=collab_thread_id,
    user_message=adaptation_prompt + "\n\nPrevious contributions: " + summary,
    agent_name="audience_adaptation"
)
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="audience_adaptation", content=adaptation_contribution)
print(f"Audience Adaptation Agent: {adaptation_contribution}\n")

# Final touches from variation agent
variation_prompt = "Suggest some unexpected twists or alternative approaches to make this social media sketch truly original."

print("Collaborative Session - Variation Agent's Turn\n")

# Get final conversation summary
if summary_tool:
    summary = summary_tool.function(thread_id=collab_thread_id)

variation_contribution = orchestrator.orchestrate(
    thread_id=collab_thread_id,
    user_message=variation_prompt + "\n\nPrevious contributions: " + summary,
    agent_name="variation_agent"
)
EphemeralMemory.store_message(thread_id=collab_thread_id, sender="variation_agent", content=variation_contribution)
print(f"Variation Agent: {variation_contribution}\n")

# Get the complete collaborative sketch
print("Complete Collaborative Comedy Sketch:\n")
if summary_tool:
    final_sketch = summary_tool.function(thread_id=collab_thread_id)
    print(final_sketch)

Stored initial challenge in memory

Collaborative Session - Concept Generator's Turn

Concept Generator: [concept_generator] ### Sketch Title: "Insta-Reality"

#### Characters:
1. **Influencer Ida** - A social media influencer with an exaggerated online persona.
2. **Follower Fran** - Ida's devoted follower who tries to imitate her every move.
3. **Reality Ralph** - Ida's partner who is exhausted by her constant need for social media validation.
4. **Narrator** - Occasionally steps in to provide humorous commentary.

#### Setting:
The stage is divided into two sections: one side represents Ida's filtered Instagram world (where everything looks perfect), and the other side represents her messy real life.

---

#### Scene 1: Introducing Influencer Ida

*(Lights up on a glamorous set representing Instagram)*

**Narrator:** (starts with a dramatic tone) Welcome to the fabulous world of Influencer Ida, where every moment is a picture-perfect paradise!

*(Ida enters, posing exaggeratedly wit

KeyboardInterrupt: 

## Advanced Multi-Agent Interaction

Now let's demonstrate dynamic agent selection using the classifier.

In [ ]:
# Create a message that doesn't explicitly specify which agent should handle it
ambiguous_query = """I need help creating a sketch about awkward family gatherings during 
holidays that would work well in a short-form video format. What would be a good approach?"""

# Create a new thread ID for this query
ambiguous_thread_id = "sketch_ambiguous_001"

# Let the orchestrator determine which agent should handle this query
print("Demonstrating dynamic agent selection with classifier...\n")
response = orchestrator.orchestrate(
    thread_id=ambiguous_thread_id,
    user_message=ambiguous_query
    # Note: we're not specifying an agent_name, so the classifier will choose
)

print(f"Response from the most appropriate agent:\n{response}")

# Pipeline


In [13]:
# This cell takes user input for a comedy sketch and stores the final script

import os
from IPython.display import Markdown, display
from datetime import datetime

# Create output and logs directories if they don't exist
os.makedirs("output", exist_ok=True)
os.makedirs("logs", exist_ok=True)

def create_sketch(topic, audience="YouTube", format_type="video"):
    """
    Create a comedy sketch using the multi-agent system
    
    Parameters:
    topic (str): The main topic or theme for the comedy sketch
    audience (str): Target audience platform (e.g., YouTube, TikTok, live theater)
    format_type (str): Format of the sketch (e.g., video, audio, text)
    
    Returns:
    str: The final comedy sketch script
    """
    # Create a new thread ID for this request
    thread_id = f"sketch_request_{topic.replace(' ', '_').lower()}"
    
    # Initialize logging
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_file_path = f"logs/{topic.replace(' ', '_').lower()}_{timestamp}_log.md"
    
    def log_to_file(step, message_type, content):
        """Helper function to log to the file"""
        with open(log_file_path, "a") as log_file:
            log_file.write(f"## {step}: {message_type}\n\n")
            log_file.write(f"{content}\n\n")
            log_file.write("---\n\n")
    
    # Start logging with header
    with open(log_file_path, "w") as log_file:
        log_file.write(f"# Comedy Sketch Creation Log: {topic}\n\n")
        log_file.write(f"- **Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        log_file.write(f"- **Topic:** {topic}\n")
        log_file.write(f"- **Audience:** {audience}\n")
        log_file.write(f"- **Format:** {format_type}\n\n")
        log_file.write("---\n\n")
    
    # Format the user request
    user_request = f"I need a comedy sketch about {topic}, suitable for a {audience} audience in {format_type} format."
    
    # Log the initial request
    log_to_file("Initial Request", "User Input", user_request)
    
    # Store user request in memory
    EphemeralMemory.store_message(thread_id=thread_id, sender="user", content=user_request)
    print(f"Creating a comedy sketch about {topic} for {audience}...\n")
    
    # Step 1: Concept Generation
    concept_task = f"""Generate 3 unique comedy sketch concepts about {topic}. 
    Each concept should include a brief premise, key characters, and the central comedic tension. 
    Make the concepts fresh, engaging, and suitable for a {audience} audience."""
    
    # Log the concept generation prompt
    log_to_file("Step 1", "Prompt to Concept Generator", concept_task)
    
    print("Step 1: Generating concepts...")
    concept_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=concept_task,
        agent_name="concept_generator"
    )
    
    # Log the concept generator's response
    log_to_file("Step 1", "Response from Concept Generator", concept_response)
    
    # Step 2: Dialogue Development
    dialogue_task = f"""Based on the concepts provided, choose the most promising one and develop 
    engaging dialogue for it. Create witty one-liners, comedic callbacks, and ensure proper 
    setup and delivery. Focus on making the dialogue sound natural while maximizing humor."""
    
    print("Step 2: Developing dialogue...")
    summary = summary_tool.function(thread_id=thread_id) if summary_tool else ""
    
    # Log the dialogue development prompt with summary
    full_dialogue_prompt = dialogue_task + "\n\nRefer to the previous concepts: " + summary
    log_to_file("Step 2", "Prompt to Dialogue Refiner", full_dialogue_prompt)
    
    dialogue_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_dialogue_prompt,
        agent_name="dialogue_refiner"
    )
    
    # Log the dialogue refiner's response
    log_to_file("Step 2", "Response from Dialogue Refiner", dialogue_response)
    
    # Step 3: Structure & Timing
    structure_task = f"""Based on the concept and dialogue developed so far, create a structured 
    sketch outline with proper comedic escalation, beats, and timing cues. Ensure the sketch 
    flows naturally, building to a satisfying comedic climax. Include specific guidance on 
    how to pace the performance for maximum comedic impact."""
    
    print("Step 3: Creating structure and timing...")
    summary = summary_tool.function(thread_id=thread_id) if summary_tool else ""
    
    # Log the structure prompt with summary
    full_structure_prompt = structure_task + "\n\nRefer to the previous work: " + summary
    log_to_file("Step 3", "Prompt to Structure & Timing Agent", full_structure_prompt)
    
    structure_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_structure_prompt,
        agent_name="structure_timing"
    )
    
    # Log the structure response
    log_to_file("Step 3", "Response from Structure & Timing Agent", structure_response)
    
    # Step 4: Audience Adaptation
    adaptation_task = f"""Adapt the sketch to ensure it resonates specifically with a {audience} 
    audience. Adjust tone, references, and comedic elements to maximize appeal 
    and relatability for this audience. Suggest any platform-specific considerations 
    that would enhance the sketch's success."""
    
    print("Step 4: Adapting for audience...")
    summary = summary_tool.function(thread_id=thread_id) if summary_tool else ""
    
    # Log the adaptation prompt with summary
    full_adaptation_prompt = adaptation_task + "\n\nRefer to the previous work: " + summary
    log_to_file("Step 4", "Prompt to Audience Adaptation Agent", full_adaptation_prompt)
    
    adaptation_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_adaptation_prompt,
        agent_name="audience_adaptation"
    )
    
    # Log the adaptation response
    log_to_file("Step 4", "Response from Audience Adaptation Agent", adaptation_response)
    
    # Step 5: Creative Variations
    variation_task = """Based on the sketch developed so far, provide 2-3 creative variations or 
    unexpected twists that could make the comedy more unique and less predictable. Suggest 
    alternative endings, subversions of expected tropes, or surprising character choices 
    that maintain the core concept while adding originality."""
    
    print("Step 5: Adding creative variations...")
    summary = summary_tool.function(thread_id=thread_id) if summary_tool else ""
    
    # Log the variation prompt with summary
    full_variation_prompt = variation_task + "\n\nRefer to the previous work: " + summary
    log_to_file("Step 5", "Prompt to Variation Agent", full_variation_prompt)
    
    variation_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_variation_prompt,
        agent_name="variation_agent"
    )
    
    # Log the variation response
    log_to_file("Step 5", "Response from Variation Agent", variation_response)
    
    # Get the complete sketch history
    complete_sketch = summary_tool.function(thread_id=thread_id) if summary_tool else ""
    
    # Extract the final script by isolating the dialogue part
    # This is a simplified extraction - in practice you might want more complex parsing
    final_script = dialogue_response
    
    # Format the final script with any adaptations and selected variations
    sketch_title = f"Comedy Sketch: {topic.title()}"
    formatted_script = f"""# {sketch_title}

## Overview
A comedy sketch about {topic} created for {audience}.

## Script
{final_script}

## Production Notes
### Structure & Timing
{structure_response}

### Audience Adaptation
{adaptation_response}

### Possible Variations
{variation_response}
"""
    
    # Save the script to a file
    filename = f"output/{topic.replace(' ', '_').lower()}_sketch.md"
    with open(filename, "w") as f:
        f.write(formatted_script)
    
    # Log the final output
    log_to_file("Final Output", "Complete Comedy Sketch", formatted_script)
    
    print(f"\nSketch completed! Saved to {filename}")
    print(f"Complete log saved to {log_file_path}")
    
    # Display a preview
    display(Markdown(f"## Preview of '{sketch_title}'\n\n```\n{final_script[:500]}...\n```\n\n[Full script saved to {filename}]\n[Complete log saved to {log_file_path}]"))
    
    return final_script

In [14]:
# Run this cell to create your custom comedy sketch!

# Replace these values with your desired parameters
topic = "Indian Wedding"  # The main topic/theme for the comedy sketch
audience = "YouTube"  # Target platform (YouTube, TikTok, Netflix, live theater, etc.)
format_type = "2 minute video"  # Format (video, audio, live performance, etc.)

# Create the sketch
final_script = create_sketch(
    topic=topic,
    audience=audience, 
    format_type=format_type
)

Creating a comedy sketch about Indian Wedding for YouTube...

Step 1: Generating concepts...
Step 2: Developing dialogue...
Step 3: Creating structure and timing...
Step 4: Adapting for audience...
Step 5: Adding creative variations...

Sketch completed! Saved to output/indian_wedding_sketch.md
Complete log saved to logs/indian_wedding_2025-03-16_12-25-04_log.md


## Preview of 'Comedy Sketch: Indian Wedding'

```
[dialogue_refiner] The dialogue for "The Overzealous Wedding Planner" has been developed and stored successfully! Here it is for your reference:

### The Overzealous Wedding Planner

**Characters:**
- Pinky the Wedding Planner
- Mr. Sharma
- Mrs. Sharma
- Sia Sharma
- Rohan Verma

**Setup:**
The sketch opens with Pinky excitedly showing Mr. and Mrs. Sharma and Sia a vision board filled with extravagant ideas for the wedding. Sia looks overwhelmed while her parents seem skeptical.

**Scene 1:**

...
```

[Full script saved to output/indian_wedding_sketch.md]
[Complete log saved to logs/indian_wedding_2025-03-16_12-25-04_log.md]